In [66]:
import os
import ast
import importlib
import importlib.util
import sys

"""
DynamicModuleModifier allows you to interact with Python module files dynamically.

Usage:
1. Initialize an instance of DynamicModuleModifier to work with a Python module file.

Example:
```python
modifier = DynamicModuleModifier('my_module.py', source='def my_function():\n    pass')
```

2. Retrieve attributes (functions or classes) defined in the loaded module using the __getitem__ method.

Example:
```python
my_function = modifier['my_function']
```

3. Replace or add attributes (functions or classes) in the module using the __setitem__ method.

Example:
```python
modifier['my_function'] = 'def my_function():\n    print("Hello, world!")'
```

4. Replace the entire content of the module with new code using the replace_module_content method.

Example:
```python
new_code = 'def new_function():\n    print("This is the new function.")'
modifier.replace_module_content(new_code)
```
"""


class DynamicModuleModifier:
    def __init__(self, filepath, source=None):
        self.filepath = filepath
        self._ensure_module_exists(source)
        self.module = self._load_module()

    def _ensure_module_exists(self, source):
        if not os.path.exists(self.filepath):
            with open(self.filepath, "w") as f:
                if source:
                    f.write(source)
                else:
                    f.write("# Auto-generated module\n")

    def __getitem__(self, key):
        return getattr(self.module, key)

    def __setitem__(self, key, new_item):
        # Read the existing code
        with open(self.filepath, "r") as f:
            old_code = f.read()
        old_ast = ast.parse(old_code)

        # Convert string to AST if necessary
        if isinstance(new_item, str):
            new_item_ast = ast.parse(new_item).body[0]
        else:
            new_item_ast = new_item

        # Determine whether to replace an existing item or add a new one
        if isinstance(new_item_ast, (ast.FunctionDef, ast.AsyncFunctionDef)):
            existing_indices = [
                i
                for i, node in enumerate(old_ast.body)
                if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef))
                and node.name == key
            ]
        elif isinstance(new_item_ast, ast.ClassDef):
            existing_indices = [
                i
                for i, node in enumerate(old_ast.body)
                if isinstance(node, ast.ClassDef) and node.name == key
            ]
        else:
            raise ValueError("The new item must be a function or class definition.")

        if existing_indices:
            # Replace existing
            old_ast.body[existing_indices[0]] = new_item_ast
        else:
            # Add new
            old_ast.body.append(new_item_ast)

        # Compile and execute the modified AST to update the module
        new_code = compile(old_ast, filename=self.filepath, mode="exec")
        exec(new_code, self.module.__dict__)

        # Write back the modified code to the file
        with open(self.filepath, "w") as f:
            f.write(ast.unparse(old_ast))

    def _load_module(self):
        module_name = self.filepath.replace(".py", "")
        if module_name in sys.modules:
            del sys.modules[module_name]

        spec = importlib.util.spec_from_file_location(module_name, self.filepath)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        return module

    def replace_module_content(self, new_content):
        with open(self.filepath, "w") as f:
            f.write(new_content)
        self.module = self._load_module()

In [67]:
# test_code.py

# Create the initial module file
with open("my_module.py", "w") as f:
    f.write(
        '''
def my_function(x):
    """This is the docstring for my_function."""
    return x * 2
'''
    )

# Initialize the DynamicModuleModifier
module_modifier = DynamicModuleModifier("my_module.py")

# Test original function
my_function = module_modifier["my_function"]
assert my_function(5) == 10
assert my_function.__doc__ == "This is the docstring for my_function."
print(my_function.__doc__)

# Modify the function using new AST
new_function_ast = ast.parse(
    '''
def my_function(x):
    """This is the modified docstring."""
    return x * 3
'''
).body[0]

module_modifier["my_function"] = new_function_ast

# Reload the module and test again
module_modifier = DynamicModuleModifier("my_module.py")
my_function = module_modifier["my_function"]
assert my_function(5) == 15
assert my_function.__doc__ == "This is the modified docstring."
print(my_function.__doc__)


# Replace the entire module content and test
new_module_content = '''
def another_function(y):
    """New function after replacing module."""
    return y ** 2
'''
module_modifier.replace_module_content(new_module_content)

# Reload the module and test the new function
module_modifier = DynamicModuleModifier("my_module.py")
another_function = module_modifier["another_function"]
assert another_function(4) == 16
assert another_function.__doc__ == "New function after replacing module."


new_class = '''
class MyClass:
    """This is the docstring for MyClass."""
    def __init__(self, x):
        """This is the docstring for __init__."""
        self.x = x

    def my_method(self, y):
        """This is the docstring for my_method."""
        return self.x + y
'''
module_modifier["MyClass"] = new_class

This is the docstring for my_function.
This is the modified docstring.


In [39]:
# print my_module.py
print(open("my_module.py").read())


def another_function(y):
    """New function after replacing module."""
    return y ** 2



In [40]:
mm = module_modifier
mm.module

# show mm.module contents without using the file
mm.module.another_function(4)

16

In [43]:
dmm = DynamicModuleModifier
initial_source = '''
def my_function(x):
    """This is the docstring for my_function."""
    return x * 2
'''
mm = dmm("new_module.py", initial_source)
mm.module

<module 'new_module' from '/Users/candacechatman/new_module.py'>

In [59]:
# Initialize DynamicModuleModifier and test to ensure correct behavior
dmm = DynamicModuleModifier("AgentSystem.py")
assert (
    hasattr(dmm.module, "argparse") == False
), "Initial module should not have argparse"

# Define our initial source code, ensuring its existence and load into our DynamicModuleModifier
initial_source = """
import argparse
import openai
import yaml
import os
import time
import json
from typing import List
import re
"""

dmm.replace_module_content(initial_source)
assert hasattr(
    dmm.module, "argparse"
), "Module should have argparse after initial source"

# Add more code to the module and verify
load_schema_code = """
def load_schema(file_path: str = 'bounded_context.yaml') -> dict:
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            return yaml.safe_load(file)
    return {}
"""
load_schema_ast = ast.parse(load_schema_code).body[0]
dmm["load_schema"] = load_schema_ast
assert hasattr(dmm.module, "load_schema"), "Module should have load_schema"

# Add another function and verify
save_schema_code = """
def save_schema(schema: dict, file_path: str = 'bounded_context.yaml') -> None:
    with open(file_path, 'w') as file:
        yaml.safe_dump(schema, file)
"""
save_schema_ast = ast.parse(save_schema_code).body[0]
dmm["save_schema"] = save_schema_ast
assert hasattr(dmm.module, "save_schema"), "Module should have save_schema"

# Add the closed-loop agent system and verify
closed_loop_agent_system_code = """
def closed_loop_agent_system(num_iterations: int, delay: int) -> None:
    schema = load_schema()
    for _ in range(num_iterations):
        time.sleep(delay)
"""

closed_loop_agent_system_ast = ast.parse(closed_loop_agent_system_code).body[0]
dmm["closed_loop_agent_system"] = closed_loop_agent_system_ast
assert hasattr(
    dmm.module, "closed_loop_agent_system"
), "Module should have closed_loop_agent_system"

# Verify the module code is as expected
with open("AgentSystem.py", "r") as f:
    code = f.read()

assert "argparse" in code, "The code must contain argparse"
assert "load_schema" in code, "The code must contain load_schema"
assert "save_schema" in code, "The code must contain save_schema"
assert (
    "closed_loop_agent_system" in code
), "The code must contain closed_loop_agent_system"

print("All assertions passed. AgentSystem.py has been successfully modified.")

AssertionError: Module should have load_schema

In [53]:
%tb

SystemExit: 2